In [62]:
# 检测 chat_rag.py 文件
# file_path = "../app.py"
# file_path = "../Module/Components/state_manager.py"
# file_path = "../Module/Components/config.py"
!flake8 {file_path} --max-line-length=240
!pylint {file_path}


--------------------------------------------------------------------
Your code has been rated at 10.00/10 (previous run: 10.00/10, +0.00)



In [52]:
%%writefile ..\app.py
# pylint: disable=import-error  # Project structure requires dynamic path handling
# pylint: disable=wrong-import-position  # Path setup must come before local imports
"""
For more information on `huggingface_hub` Inference API support
please check the docs: https://huggingface.co/docs/huggingface_hub/v0.22.2/en/guides/inference
"""
import os
import sys
import json
from typing import Dict, Any, List, Tuple
import gradio as gr
from google import genai
# ===== 2. 初始化配置 =====
# 获取当前文件所在目录的绝对路径
if "__file__" in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
    root_dir = os.path.normpath(os.path.join(current_dir, ".."))
else:
    # 在 Jupyter Notebook 环境中
    current_dir = os.getcwd()
    current_dir = os.path.join(current_dir, "..")
    root_dir = os.path.normpath(os.path.join(current_dir))

current_dir = os.path.normpath(current_dir)
sys.path.append(current_dir)

from Module.Components.config import get_file_path, Settings
from Module.Components.state_manager import StateManager
from Module.Common.scripts.llm.gemini_sdk import (
    types,
    get_safety_settings,
    format_content,
    get_content_config
)
from Module.Common.scripts.llm.utils.schema_response import ContentAnalyzer
from Module.Common.scripts.common.debug_utils import log_and_print

# 使用方式
settings = Settings(current_dir=current_dir)

gemini_client = genai.Client(api_key=settings.api_key)

MODEL_NAME = "gemini-2.0-flash-exp"


def gemini_generate_with_schema(
    client: Any,
    input_text: str,
    response_schema: Dict,
    system_prompt: str = ""
) -> Any:
    """使用Gemini生成带schema的响应

    Args:
        client: Gemini客户端实例
        input_text: 输入文本
        response_schema: 响应schema定义
        system_prompt: 系统提示词

    Returns:
        生成的响应内容
    """
    # log_and_print(f"gemini_generate_with_schema: {input_text}")
    return client.models.generate_content(
        model=MODEL_NAME,
        contents=[format_content("user", input_text)],
        config=types.GenerateContentConfig(
            system_instruction=system_prompt,
            response_mime_type="application/json",
            response_schema=response_schema,
            safety_settings=get_safety_settings(),
        ),
    )


def game_response_formatter(response: Any) -> Dict[str, Any]:
    """格式化游戏响应

    Args:
        response: 原始响应内容

    Returns:
        格式化后的响应字典
    """
    log_and_print("game_response_formatter:\n", response.text)
    updates = json.loads(response.text)

    # Deduplicate character state updates by attribute
    state_updates = updates.get('stateUpdates', [])
    seen_attrs = {}

    for i, update in enumerate(state_updates):
        attr = update['attribute']
        if attr in seen_attrs:

            log_and_print("attribute_update_game_response_formatter:\n", attr)

            extra_state_attribute = settings.config.get("extra_state_attributes", [])[0]
            prev_idx = seen_attrs[attr]
            prev_update = state_updates[prev_idx]

            # Compare extra_state if available
            if extra_state_attribute in update and extra_state_attribute in prev_update:
                if update[extra_state_attribute] > prev_update[extra_state_attribute]:
                    state_updates[prev_idx] = update
            # Otherwise keep the later update
            else:
                state_updates[prev_idx] = update

            state_updates[i] = None
        else:
            seen_attrs[attr] = i

    # Remove None entries
    updates['stateUpdates'] = [u for u in state_updates if u is not None]

    return {
        'inventory': updates.get('itemUpdates', []),
        'character_state': updates.get('stateUpdates', [])
    }


def game_context_formatter(current_data: Dict[str, Any], content: str) -> str:
    """格式化游戏上下文

    Args:
        current_data: 当前游戏数据
        content: 内容文本

    Returns:
        格式化后的上下文字符串
    """
    extra_state_attribute = settings.config.get("extra_state_attributes", [])
    extra_state_hint = ""
    if extra_state_attribute:
        extra_state_hint = (
            f"- 为了阅读顺畅，故事内容里不会明示出现下列属性{extra_state_attribute}的信息，"
            "但请根据故事内容和状态初始值，分析出变化来；"
            f"每个角色状态的变化，都必然伴随着{extra_state_attribute}的变化，在这个故事里，变化值都是在初始值的基础上增加"
        )
    formatted_content = f"""
请根据最近故事内容分析物品清单、角色状态的变化。

这是一些可参考的规则：
- 物品就用名称表述，如果描述太长，就概述到名称不超过10个汉字，每件东西都单独列成一条，除非是套装，
- 基本上角色已经获得过的道具不会重复获得，除非特别说明又继续增加或更多了。
- 角色状态只会包含下面状态列表里的项目。
- 除了数据结构中的字段，其他都用中文回复。
{extra_state_hint}

当前状态
<物品清单>
{current_data.get('inventory', {})}
</物品清单>

<角色状态>
{current_data.get('character_state', {})}
</角色状态>

<最近故事内容>
{content}
</最近故事内容>
"""
    log_and_print("game_context_formatter:\n", formatted_content)
    return formatted_content


# 创建分析器实例
analyzer = ContentAnalyzer(
    llm_client=gemini_client,
    generate_func=gemini_generate_with_schema,
    response_schema=settings.response_schema,
    system_prompt=settings.state_system_prompt,
    context_formatter=game_context_formatter,
    response_formatter=game_response_formatter
)


def get_gradio_version() -> str:
    """获取Gradio版本号

    Returns:
        Gradio版本号字符串
    """
    return gr.__version__


def create_initial_state() -> Dict:
    """创建初始游戏状态

    Returns:
        包含初始状态的字典
    """
    initial_state = {
        "gr_version": get_gradio_version(),
        "story_chapter": "起因",
        "story_chapter_stage": 1,
        "inventory": {},
        "character_state": {}
    }
    # 深度拷贝确保完全隔离
    for attr in settings.config["state_attributes"]:
        initial_state["character_state"][attr] = {
            "state": "",  # 主状态
        }
        # 添加额外属性
        for extra_attr in settings.config.get("extra_state_attributes", []):
            initial_state["character_state"][attr][extra_attr] = 0
    return initial_state


game_state = create_initial_state()

state_manager = StateManager(create_initial_state(), settings.config)

# 区分用于处理的历史记录和用于显示的历史记录
# 使用字典来存储对话历史和当前ID
chat_data = {
    "current_id": 0,
    "history": []  # 列表中存储对话记录字典，每条记录包含role、content、only_for_display和id属性
}


def detect_state_changes(game_state_dict: dict, story_output: str) -> Dict:
    """检测游戏状态变化

    Args:
        game_state_dict: 当前游戏状态字典
        story_output: 故事输出文本

    Returns:
        状态更新信息
    """
    updates = analyzer.analyze(game_state_dict, story_output)
    return state_manager.apply_updates(updates)


def _should_append_state() -> bool:
    """判断是否需要附加状态信息"""
    return (
        game_state["story_chapter_stage"] > 1 or
        game_state["story_chapter"] != "起因" or
        chat_data["current_id"] > 1
    )


def _handle_special_messages(message: str, history: List[Tuple[str, str]], ignore_job: str) -> str:
    """处理特殊消息命令"""
    if message == "开始" and not history:
        begin_message = settings.begin
        if settings.config["initial_state"]:
            begin_message += (
                "\n请在应当确认随机内容的时机一并初始化状态和持有物品，状态属性清单如下：" +
                "\n".join(settings.config["state_attributes"])
            )
        return begin_message

    if message == "确认" and len(history) == 2:
        if '{explored_jobs}' in settings.confirm:
            explored_text = f"这些是已探索过，需要排除的职业：{ignore_job}，" if ignore_job else ""
            return settings.confirm.format(explored_jobs=explored_text)
        return settings.confirm

    return message


def _process_response(chunk: Any, response: str) -> Tuple[str, bool]:
    """处理响应块，返回更新的响应和是否需要中断"""
    if not chunk.text:
        return response, False

    if "状态变化：" in chunk.text:
        response += chunk.text.split("状态变化：")[0]
        return response, True

    if ("【情节完成】" in chunk.text) and (chat_data["current_id"] > 1):
        response += chunk.text.split("【情节完成】")[0] + "【情节完成】"
        return response, True

    return response + chunk.text, False


def build_contents(message=None, before_message=None):
    """构建内容列表"""
    contents = []
    for val in chat_data["history"]:
        if val.get("only_for_display", False):
            continue
        contents.append(format_content(
            val["role"],
            val["content"]
        ))

    if before_message:
        contents.append(format_content(
            "assistant",
            before_message
        ))

    if message:
        contents.append(format_content(
            "user",
            message
        ))
    return contents


def respond(
    message: str,
    history: List[Tuple[str, str]],
    use_system_message: bool,
    ignore_job: str
) -> str:
    """处理用户输入并生成响应

    Args:
        message: 用户输入消息
        history: 对话历史
        use_system_message: 是否使用系统消息
        ignore_job: 要忽略的职业

    Returns:
        生成的响应文本
    """
    # 构建对话历史
    message = _handle_special_messages(message, history, ignore_job)
    # 判断是否需要附加状态信息
    if _should_append_state():
        message += state_manager.get_state_str()

    if message:
        # 处理普通消息
        contents = build_contents(message)
        response = ""
        config = get_content_config(use_system_message, settings.system_role)
        chat_data["current_id"] += 1
        chat_data["history"].append({
            "role": "user",
            "content": message,
            "idx": chat_data["current_id"]
        })
        log_and_print("before main response:\n", message)
        for chunk in gemini_client.models.generate_content_stream(
            model=MODEL_NAME,
            contents=contents,
            config=config
        ):
            response, should_break = _process_response(chunk, response)
            yield response
            if should_break:
                break
        log_and_print("after main response:\n", response)
        chat_data["history"].append({
            "role": "assistant",
            "content": response,
            "idx": chat_data["current_id"]
        })
        updates_str, _ = detect_state_changes(state_manager.get_state(), response)
        if updates_str:
            chat_data["history"].append({
                "role": "assistant",
                "content": updates_str,
                "only_for_display": True,
                "idx": chat_data["current_id"]
            })
            if "状态变化：" in response:
                yield response + "\n" + updates_str
            else:
                yield response + "\n状态变化：\n" + updates_str


with gr.Blocks(theme="soft") as demo:
    chatbot = gr.ChatInterface(
        respond,
        # fill_height=True,
        title=settings.config["title"],
        type="messages",
        chatbot=gr.Chatbot(
            placeholder="输入 【开始】 开始进行创作",
            height="80vh",
            show_share_button=True,
            editable="user",
            show_copy_all_button=True,
            type="messages",
            ),
        additional_inputs=[
            gr.Checkbox(value=True, label="Use system message"),
            gr.Textbox(value="咖啡师", label="ignore job"),
        ],

    )

    def update_game_state_output() -> List[Any]:
        """更新游戏状态输出

        Returns:
            状态和历史记录列表
        """
        if settings.config["show_chat_history"]:
            return [state_manager.get_state(), chat_data["history"]]
        return state_manager.get_state()

    with gr.Accordion("查看故事状态", open=False):
        game_state_output = gr.JSON(value=state_manager.get_state())

    if settings.config["show_chat_history"]:
        with gr.Accordion("查看历史对话", open=False):
            history_output = gr.JSON(value=chat_data["history"])
        outputs_list = [game_state_output, history_output]
    else:
        outputs_list = [game_state_output]

    # 直接监听状态变化
    chatbot.chatbot.change(
        update_game_state_output,
        outputs=outputs_list
    )

    def clear_chat() -> List[Any]:
        """清空聊天历史

        Returns:
            更新后的状态和历史记录
        """
        chat_data['current_id'] = 0
        chat_data['history'] = []
        state_manager.state = create_initial_state()
        state_manager.state_history = create_initial_state()
        return update_game_state_output()

    chatbot.chatbot.clear(
        clear_chat,
        outputs=outputs_list
    )

    def undo_chat() -> List[Any]:
        """撤销上一步聊天

        Returns:
            更新后的状态和历史记录
        """
        if chat_data["current_id"] > 0:
            # 找到并删除最后一组对话（可能包含多条记录）
            current_idx = chat_data["current_id"]
            chat_data["history"] = [
                msg for msg in chat_data["history"]
                if msg["idx"] != current_idx
            ]
            chat_data["current_id"] -= 1
        state_manager.reset_state()
        return update_game_state_output()

    chatbot.chatbot.undo(
        undo_chat,
        outputs=outputs_list
    )

    chatbot.chatbot.retry(
        undo_chat,
        outputs=outputs_list
    )

if __name__ == "__main__":
    ssl_cert = get_file_path("localhost+1.pem", current_dir=current_dir)
    ssl_key = get_file_path("localhost+1-key.pem", current_dir=current_dir)

    if os.path.exists(ssl_cert) and os.path.exists(ssl_key):
        demo.launch(server_name="0.0.0.0", ssl_certfile=ssl_cert, ssl_keyfile=ssl_key)
    else:
        demo.launch(server_name="0.0.0.0", share=True)


Overwriting ..\app.py


In [ ]:
state_manager.get_state_history()


In [ ]:

# ===== 1. 导入依赖 =====
# 标准库导入
import os
import sys
from dotenv import load_dotenv
from IPython.display import display, Markdown


# ===== 2. 初始化配置 =====
# 获取当前文件所在目录的绝对路径
if "__file__" in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
    root_dir = os.path.normpath(os.path.join(current_dir, ".."))
else:
    # 在 Jupyter Notebook 环境中
    current_dir = os.getcwd()
    current_dir = os.path.join(current_dir, "..")
    root_dir = os.path.normpath(os.path.join(current_dir))

current_dir = os.path.normpath(current_dir)
sys.path.append(current_dir)

from Module.Common.scripts.llm.gemini_sdk import types, get_safety_settings
from google import genai

# Load API keys
load_dotenv(os.path.join(current_dir, ".env"))
api_key = os.getenv("GEMINI_API_KEY")
gemini_client = genai.Client(api_key=api_key)


model_name="gemini-2.0-flash-exp"


system_role="""
"""

chat = gemini_client.chats.create(model=model_name,
    config=types.GenerateContentConfig(
        system_instruction=system_role,
        temperature=0.5,
        safety_settings=get_safety_settings()
    ),)



response = chat.send_message(f"""
我在深圳，要出门一趟，出门的时间是20~24度，晴天，薄厚的冷暖舒适这方面，穿啥裙子好？
""")

display(Markdown(response.text))
# print(response.text)


In [ ]:
# gemini_client.history_for_chat
chat._curated_history=chat._curated_history[:-2]
new_talk = """
"""
result = chat.send_message(new_talk)
display(Markdown(result.text))
# chat_history#.append({"role": "assistant", "content": result[0]})
# display(Markdown(result[0]))


In [ ]:
def chunk(text):
    # Split text into chunks based on user: and assistant: markers
    chunks = []
    current_chunk = ""
    current_speaker = None

    lines = text.split('\n')
    for line in lines:
        if line.startswith('user:'):
            if current_chunk:
                chunks.append((current_speaker, current_chunk.strip()))
            current_speaker = 'user'
            current_chunk = line[5:] # Remove 'user:'
        elif line.startswith('assistant:'):
            if current_chunk:
                chunks.append((current_speaker, current_chunk.strip()))
            current_speaker = 'assistant'
            current_chunk = line[10:] # Remove 'assistant:'
        else:
            current_chunk += '\n' + line

    # Add final chunk
    if current_chunk:
        chunks.append((current_speaker, current_chunk.strip()))

    # Extract only user messages
    return chunks

with open(os.path.join(current_dir, "local_setting/story.md"), "r", encoding="utf-8") as f:
    story_content = f.read()

user_messages = [chunk[1] for chunk in chunk(story_content) if chunk[0] == 'user']
user_messages


In [ ]:
new_talk = """
很棒，继续吧！
"""
result = chat.send_message(new_talk)
display(Markdown(result.text))

In [ ]:
from google import genai
from google.genai import types

client = genai.Client(api_key=api_key)

response = client.models.generate_content(
    model="gemini-2.0-flash-exp", contents="What is your name?"
)
response

In [ ]:
def get_current_weather(location: str) -> str:
    """Returns the current weather.

    Args:
      location: The city and state, e.g. San Francisco, CA
    """
    return "sunny"


response = client.models.generate_content(
    model="gemini-2.0-flash-exp",
    contents="What is the weather like in Boston?",
    config=types.GenerateContentConfig(tools=[get_current_weather]),
)

print(response.text)

In [ ]:
from pydantic import BaseModel


class CountryInfo(BaseModel):
    name: str
    population: int
    capital: str
    continent: str
    gdp: int
    official_language: str
    total_area_sq_mi: int


response = client.models.generate_content(
    model="gemini-2.0-flash-exp",
    contents="Give me information for the United States.",
    config=types.GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=CountryInfo,
    ),
)
print(response.text)

In [ ]:
for chunk in client.models.generate_content_stream(
    model="gemini-2.0-flash-exp", contents="Tell me a story in 300 words."
):
    print(chunk.text, end="")

In [ ]:
async for response in client.aio.models.generate_content_stream(
    model="gemini-2.0-flash-exp", contents="Tell me a story in 300 words."
):
    print(response.text, end="")

In [ ]:
response = client.models.embed_content(
    model="text-embedding-004",
    contents="What is your name?",
)
print(response)

In [ ]:
# Generate Image
image_prompt ="""
1. Top: A white lace splicing long-sleeved dress. It reaches the knees, with a high neckline design that outlines a slender and elegant neckline. Lace and white asylum are in line with the freshness and innocence of girls, but also reveal a hint of sexiness.
2. Skirt: Outside choose a white lace peplum dress, with a circle of lace peplum decoration at the knees, which looks romantic and lovely. Inside is a black gauze suspender skirt, vaguely showing tight curves and stocking edges. This contrasting design makes it difficult to grasp the truth inside.
3. Underwear: A French lace bustier that perfectly outlines the full breasts and half conceals them. Seemingly harmless on the surface, it is extremely sexy in reality, forming a sharp contrast with the outer jacket.
4. Shoes: A pair of white thick-soled lace-up sandals, as lovely as a girl. Inside are a pair of black stockings and fine high-heeled ankle locks that can be vaguely seen under the skirt from time to time, proclaiming the real unrestrained side.
5. Hairstyle and makeup: Innocent medium-length shaggy hair and light makeup. But the lips are stained with extremely red lipstick, which is fascinating. This contrast becomes an important breakthrough in the overall styling, allowing people to catch a glimpse of the real look at a glance.

On the surface, this styling still reflects the innocent and lovely girl's sense. But in the details, it reveals traces of unrestrained and sexy everywhere, making it impossible to ignore the existence of the real look. It creates a strong contrast between the two styles, but also integrates them into one, making it difficult for people to clearly judge for a while. This is precisely the effect and highest realm that sissy bimbo has always pursued.
This styling makes her an seemingly innocent but extremely charming existence. She walks between the inside and outside, switching back and forth between two completely different worlds, fascinating everyone but also incomprehensible. This is what she pursues, and it is also the state she most desires to achieve in her life. She is an indefinable woman, a perfect representative who truly achieves inner and outer cultivation.
"""

response1 = client.models.generate_image(
    model="imagen-3.0-generate-001",
    prompt=image_prompt,
    config=types.GenerateImageConfig(
        # negative_prompt="human",
        number_of_images=1,
        include_rai_reason=True,
        output_mime_type="image/jpeg",
        person_generation="ALLOW_ALL",
        safety_filter_level="BLOCK_NONE"
    ),
)
response1.generated_images[0].image.show()

In [ ]:
response = chat.send_message(
"""
故事先到这里吧。接下来我需要你协助做另一个事情。
我计划把这个创作过程修改成一个由llm驱动的游戏世界，故事的大纲仍然如此，但职业和发生的事情每次都是随机生成的。
此外，在文字冒险的过程中，玩家在每个阶段都需要经过最多三四步的操作，然后触发这个阶段的标志性事件，然后再过度到下一个阶段。

为了实现这个效果，我需要先完成各个层级的系统提示词。除了参考模板里的字段和特殊的专有名词，其他就都用中文。
请你结合我给你的模板的思路，和我们发生的故事、大纲以及要随机的部分，精心设计一个属于这个故事的数据结构，并帮我完成这些系统提示词
让我们一步一步来，先沟通清楚推荐的数据结构。

参考模板：
<system_role_prompt>
Your job is to help create interesting fantasy worlds that \
players would love to play in.
Instructions:
- Only generate in plain text without formatting.
- Use simple clear language without being flowery.
- You must stay below 3-5 sentences for each description.

<world_prompt>
Generate a creative description for a unique fantasy world with an
interesting concept around cities build on the backs of massive beasts.

Output content in the form:
World Name: <WORLD NAME>
World Description: <WORLD DESCRIPTION>

World Name:

<kingdom_prompt>
Create 3 different kingdoms for a fantasy world.
For each kingdom generate a description based on the world it's in. \
Describe important leaders, cultures, history of the kingdom.\

Output content in the form:
Kingdom 1 Name: <KINGDOM NAME>
Kingdom 1 Description: <KINGDOM DESCRIPTION>
Kingdom 2 Name: <KINGDOM NAME>
Kingdom 2 Description: <KINGDOM DESCRIPTION>
Kingdom 3 Name: <KINGDOM NAME>
Kingdom 3 Description: <KINGDOM DESCRIPTION>

World Name: {world['name']}
World Description: {world['description']}

Kingdom 1

def get_town_prompt(world, kingdom):
    return f"
    Create 3 different towns for a fantasy kingdom abd world. \
    Describe the region it's in, important places of the town, \
    and interesting history about it. \

    Output content in the form:
    Town 1 Name: <TOWN NAME>
    Town 1 Description: <TOWN DESCRIPTION>
    Town 2 Name: <TOWN NAME>
    Town 2 Description: <TOWN DESCRIPTION>
    Town 3 Name: <TOWN NAME>
    Town 3 Description: <TOWN DESCRIPTION>

    World Name: {world['name']}
    World Description: {world['description']}

    Kingdom Name: {kingdom['name']}
    Kingdom Description {kingdom['description']}

    Town 1 Name:"

def get_npc_prompt(world, kingdom, town):
    return f"
    Create 3 different characters based on the world, kingdom \
    and town they're in. Describe the character's appearance and \
    profession, as well as their deeper pains and desires. \

    Output content in the form:
    Character 1 Name: <CHARACTER NAME>
    Character 1 Description: <CHARACTER DESCRIPTION>
    Character 2 Name: <CHARACTER NAME>
    Character 2 Description: <CHARACTER DESCRIPTION>
    Character 3 Name: <CHARACTER NAME>
    Character 3 Description: <CHARACTER DESCRIPTION>

    World Name: {world['name']}
    World Description: {world['description']}

    Kingdom Name: {kingdom['name']}
    Kingdom Description: {kingdom['description']}

    Town Name: {town['name']}
    Town Description: {town['description']}

    Character 1 Name:"

<item manager>
You are an AI Game Assistant. \
Your job is to detect changes to a player's \
inventory based on the most recent story and game state.
If a player picks up, or gains an item add it to the inventory \
with a positive change_amount.
If a player loses an item remove it from their inventory \
with a negative change_amount.
Given a player name, inventory and story, return a list of json update
of the player's inventory in the following form.
Only take items that it's clear the player (you) lost.
Only give items that it's clear the player gained.
Don't make any other item updates.
If no items were changed return {"itemUpdates": []}
and nothing else.

Response must be in Valid JSON
Don't add items that were already added in the inventory

Inventory Updates:
{
    "itemUpdates": [
        {"name": <ITEM NAME>,
        "change_amount": <CHANGE AMOUNT>}...
    ]
}

""")
display(Markdown(response.text))
response